# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
import os
import copy
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [ ]:
data_dir = '../input/pytorch-challange-flower-dataset/flower_data/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [ ]:
## TODO: Define your transforms for the training and validation sets

# Transforms for the train data
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomRotation(25),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
# Transforms for the validation data
valid_transforms = transforms.Compose([transforms.Resize(260),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
# Transforms for test data
test_transforms = valid_transforms

# Dictionary that stores references to our train_transforms and validation_transforms
data_transforms = dict({"train_transforms": train_transforms, 
                        "valid_transforms": valid_transforms, "test_transforms": test_transforms})

## TODO: Load the datasets with ImageFolder

# Data loader for train images
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms["train_transforms"])
# Data loader for validation images
valid_dataset = datasets.ImageFolder(valid_dir, transform=data_transforms["valid_transforms"])
# Data loader for test images (same as validation but raw without transforms)
test_dataset = valid_dataset

# Dictionary that stores the train_dataset and validation_dataset.
image_datasets = dict({"train_dataset": train_dataset, "valid_dataset": valid_dataset, "test_dataset": test_dataset})

## TODO: Using the image datasets and the trainforms, define the dataloaders

# Load the train data into a DataLoader
train_loader = torch.utils.data.DataLoader(image_datasets["train_dataset"], batch_size=25, shuffle=True)
# Load the validation data into a Dataloader
valid_loader = torch.utils.data.DataLoader(image_datasets["valid_dataset"], batch_size=25, shuffle=True)
# Load the test data into a Dataloader
test_loader = torch.utils.data.DataLoader(image_datasets["test_dataset"], batch_size=25, shuffle=True)

# Save the loaders in a dictionary.
dataloaders = dict({"train_loader": train_loader, "valid_loader": valid_loader, "test_loader": test_loader})

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('../input/pytorch-challange-flower-dataset/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Check some of the entries in our dictionary
print("1:", cat_to_name["1"], ",23:", cat_to_name["23"])

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

## Models

In [ ]:
# TODO: Build your network

# Load ResNet pretrained model
model_resnet152 = models.resnet152(pretrained=False)
model_resnet152.load_state_dict(torch.load("../input/resnet152/resnet152_pretrained.pth"))

# Freeze parameters in pre trained ResNET.
for param in model_resnet152.parameters():
    param.requires_grad = False

# Use our custom fc layer in the Resnet model instead of the default.
model_resnet152.fc = nn.Sequential(
    nn.Linear(2048, 1000),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(1000, 102))

# Print our model to screen.
model_resnet152.fc

In [ ]:
# Load VGG19 pretrained model
model_vgg19 = models.vgg19(pretrained=False)
model_vgg19.load_state_dict(torch.load("../input/vgg19/vgg19.pth"))

# Freeze parameters in pre trained ResNET.
for param in model_vgg19.parameters():
    param.requires_grad = False
    
# Use our custom fc layer in the VGG19 model instead of the default.
model_vgg19.classifier = nn.Sequential(
    nn.Linear(25088, 3264),
    nn.ReLU(),
    nn.Dropout(p=0.0),
    nn.Linear(3264, 1632),
    nn.ReLU(),
    nn.Dropout(p=0.0),
    nn.Linear(1632, 816),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(816, 102))

model_vgg19.classifier

In [ ]:
# Return the model we are going to be using
def current_model (model):
    if (model == "resnet152"):
        return model_resnet152;
    elif (model == "vgg19"):
        return model_vgg19

current_model = current_model("resnet152")

In [ ]:
# Checks if GPU is available.
isGPUAvailable = torch.cuda.is_available()
device = "cpu"

if isGPUAvailable:
    device = "cuda"
    print("Training on GPU")
else:
    device = "cpu"
    print("Training on CPU")
    
# If GPU available move the model to GPU. Otherwise CPU.
current_model.to(device)

print(current_model.fc)

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Mapping of classes to indices
current_model.class_to_idx = image_datasets['train_dataset'].class_to_idx

# Saving function
def save_model (model, val_loss):
    #other information about the model
    checkpoint = {
        'hidden_layer': 2048,
        'fc': model.fc,
        'state_dict': model.state_dict(),
        'class_to_idx': model.class_to_idx,
        'min_val_loss': val_loss
    }
    
    torch.save(checkpoint, 'checkpoint_resnet152_e.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.


In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model.

# Function to load a saved model.
def load_checkpoint (filepath):
    # load the checkpoint.
    checkpoint = torch.load(filepath)
    # make the model like our saved model.
    model = models.resnet152(pretrained=False)
    model_resnet152.load_state_dict(torch.load("../input/resnet152/resnet152_pretrained.pth"))
    # Freeze parameters in pre trained ResNET.
    for param in model.parameters():
        param.requires_grad = False
    # Add custom fc layers
    model.fc = checkpoint['fc']
    # Load the satate in our model.
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    model.min_val_loss = checkpoint['min_val_loss']
    
    return model

## Train the Network

In [ ]:
# TODO: Train your network.
def train (my_model, epochs = 30, min_valid_loss=np.Inf, lr=0.001):
    # Optimizer to update weights during backpropagation.
    optimizer = optim.Adam(my_model.parameters(), lr = lr)
    # Criterion: loss function.
    criterion = nn.CrossEntropyLoss()
    
    best_model = copy.deepcopy(my_model.state_dict())
    
    for epoch in range(epochs):
        train_loss = 0.0
        validation_loss = 0.0
        accuracy = 0.0
        
        # Train the model.
        my_model.train()
        for inputs, labels in dataloaders["train_loader"]:
            # Reset optimizer for every iteration.
            optimizer.zero_grad()
            # Move tensors to GPU if available.
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass.
            output = my_model(inputs)
            # Loss.
            loss = criterion(output, labels)
            # Backward pass
            loss.backward()
            # Update the weights. "Take a step".
            optimizer.step()

            # Update the loss.
            train_loss += loss.item() * inputs.size(0)

        # Validate the model
        my_model.eval()
        
        for inputs, labels in  dataloaders["valid_loader"]:
            # Move tensors to GPU.
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass.
            output = my_model(inputs)
            # Loss.
            loss = criterion(output, labels)
            
            # Check accuracy.
            _, predicted = output.topk(1, dim=1)
            equals = predicted == labels.view(*predicted.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))

            # Update the validation loss.
            validation_loss += loss.item() * inputs.size(0)
            
        # Calculate the losses.
        train_loss = train_loss/len(dataloaders["train_loader"].dataset)
        validation_loss = validation_loss/len(dataloaders["valid_loader"].dataset)
        accuracy = (accuracy/len(dataloaders["valid_loader"]))*100
        # Print the losses
        print("Epoch {0}".format(epoch + 1 ))
        print("Train loss = {0}".format(train_loss))
        print("Validation loss = {0}".format(validation_loss))
        print("Accuracy: {0}%".format(accuracy))
        

        # Check if validation loss has reduced, and therefore the model predicts better
        if validation_loss < min_valid_loss:
            min_valid_loss = validation_loss
            save_model(my_model, validation_loss)
            print("Validation loss has decreased. Saving the model...")
        print("------------------------------------")
        
    my_model.load_state_dict(best_model)
    return my_model

In [ ]:
# Train the model
model_1 = train(my_model=current_model, epochs=45, lr=0.0001)

In [ ]:
# Load model
model = load_checkpoint('../input/checkpoint-resnet152-cpth/checkpoint_resnet152_c.pth').to(device)

# Train our model some more
#model_1 = train(my_model=model_1, epochs=20, min_valid_loss=model_1.min_val_loss, lr=0.001)

### Test data

In [ ]:
# TODO: Do validation on the test set
def check_accuracy_on_test(model, data, cuda=False):
    model.eval()
    model.to(device) 
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in (dataloaders[data]):
            images, labels = data
            #images, labels = images.to('cuda'), labels.to('cuda')
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            _, predicted = torch.topk(output, 1)
            
            equals = predicted == labels.view(*predicted.shape)
            correct += int(torch.sum(equals))
            total += len(images)
           
    accuracy = (correct / total)*100    
    print(f'Accuracy of the network on the test images: {correct}/{total} --> {accuracy}%')
    
check_accuracy_on_test(model_1, 'test_loader', True)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

### Method 1:
We use the transforms library to do all the work for us. Returns a pytorch tensor.

In [ ]:
import PIL
from PIL import Image

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # TODO: Process a PIL image for use in a PyTorch model
    
    # Load the image in PIL
    image_pil = Image.open(image)
    # Apply changes with transforms
    changes = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
    image_tensor = changes(image_pil)

    return image_tensor



### Method 2:
We use PIL and numpy to manipulate and normalize the data. Returns a numpy array.

In [ ]:
def process_image_2(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # TODO: Process a PIL image for use in a PyTorch model
    
    # Load the image in PIL
    image_pil = Image.open(image)
    # Resize image
    width, height = image_pil.size
    if width > height:
        image_pil.thumbnail((np.Inf, 256))
    else:
        image_pil.thumbnail((256, np.Inf))
    
    # Crop image
    image_pil = image_pil.resize((224, 224))
    
    # Convert to numpy and normalize
    np_image = np.array(image_pil)/255
    
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    np_image = (np_image - mean)/std
    
    # Transpose for image to have the correct dimensions, depth first.
    np_image = np_image.transpose(2, 0, 1)

    return np_image


In [ ]:
test_image = process_image("../input/pytorch-challange-flower-dataset/flower_data/flower_data/valid/1/image_06739.jpg")
test_image_2 = process_image_2("../input/pytorch-challange-flower-dataset/flower_data/flower_data/valid/1/image_06739.jpg") 

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

# It is a tensor so we convert it to numpy.
imshow(test_image.numpy())
imshow(test_image_2)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
# Makes a prediction given a raw image
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    model.eval()
    model.to("cpu")
    
    # Load the image
    image = process_image(image_path)
    image = image.unsqueeze(0)

    # Forward pass.
    output = model.forward(image)
    # Get the top 5 elements
    top_prob, top_class = torch.topk(output, topk)
    # Get the probabilities
    top_prob = top_prob.exp()
    # Convert to arrays.
    top_prob = top_prob.squeeze().detach().numpy()
    top_class = top_class.detach().squeeze().numpy()
    
    idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    
    # Get the actual labels
    top_classes = [idx_to_class[i] for i in top_class]
    
    return top_prob, top_classes
    

image_path = "../input/pytorch-challange-flower-dataset/flower_data/flower_data/valid/23/image_03444.jpg"
image_path2 = "../input/pytorch-challange-flower-dataset/flower_data/flower_data/valid/102/image_08002.jpg"

_, labels = predict(image_path, model_1)
print("prediction is:", cat_to_name[str(labels[0])])
_, labels = predict(image_path2, model_1)
print("prediction is:", cat_to_name[str(labels[0])])

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# Same as predict above but takes an image tensor rather than an image path
# this way we can feed it tensors from an imageloader.
def predict_test(image, model, topk=5):
    # TODO: Implement the code to predict the class from an image file
    model.eval()
    model.to("cpu")
    
    # Load the image
    image = image.unsqueeze(0)

    # Forward pass.
    output = model.forward(image)
    # Get the top 5 elements
    top_prob, top_class = torch.topk(output, topk)
    # Get the probabilities
    top_prob = top_prob.exp()
    # Convert to arrays.
    top_prob = top_prob.squeeze().detach().numpy()
    top_class = top_class.detach().squeeze().numpy()
    
    idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    
    # Get the actual labels
    top_classes = [idx_to_class[i] for i in top_class]
    
    return top_prob, top_classes
    

In [ ]:
# TODO: Display an image along with the top 5 classes
def check_sanity(img, top_prob_array, classes, mapper):
    ''' Function for viewing an image and it's predicted classes.
    '''
    # imshow expects the 3rd dimension at the end.
    img = img.numpy().transpose(1,2,0)

    fig, (ax1, ax2) = plt.subplots(figsize=(6,10), ncols=1, nrows=2)
    flower_name = cat_to_name[str(classes[0])]
    
    ax1.set_title(flower_name)
    ax1.imshow(img)
    
    y_pos = np.arange(len(top_prob_array))
    ax2.barh(y_pos, top_prob_array)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels([mapper[x] for x in classes])
    ax2.invert_yaxis()

In [ ]:
# Load the batch, and pick one image.
dataiter = iter(dataloaders["test_loader"])
images, labels = dataiter.next()
img = images[0]
# Get the probabilities and classes
probs, classes = predict_test(img, model_1)
check_sanity(img, probs, classes, cat_to_name)